# Imports

In [ ]:
import os
import json
import matplotlib.pyplot as plt

In [ ]:
import matplotlib
matplotlib.rcParams['figure.facecolor'] = 'white'

In [ ]:
import pandas as pd
pd.options.display.max_columns = None

In [ ]:
%run ../utils/__init__.py
%run ../metrics/__init__.py

# Functions

In [ ]:
%run ../utils/files.py

In [ ]:
def load_metrics(run_id):
    folder = get_results_folder(run_id, assert_exists=True)
    filepath = os.path.join(folder, 'metrics.json')
    
    if not os.path.isfile(filepath):
        raise FileNotFoundError(filepath)

    with open(filepath, 'r') as f:
        return json.load(f)

In [ ]:
def load_outputs_csv(run_id):
    folder = get_results_folder(run_id, assert_exists=True)
    filepath = os.path.join(folder, 'outputs.csv')
    
    if not os.path.isfile(filepath):
        raise FileNotFoundError(filepath)
    
    return pd.read_csv(filepath)

# Compare multiple models

In [ ]:
from collections import defaultdict
import re

In [ ]:
%run ../datasets/common/__init__.py
%run ../utils/plots.py

In [ ]:
DISEASES = CXR14_DISEASES
# DISEASES = VINBIG_DISEASES
len(DISEASES)

## Load metadata

In [ ]:
%run ../models/checkpoint/__init__.py
%run ../utils/files.py

In [ ]:
# run_name = '0201_180839_cxr14_tfs-small_lr0.0001_normS_size256_sch-roc_auc-p5-f0.1_noes'
run_name = '0309_215826'
debug_run = False

In [ ]:
meta = load_metadata(RunId(run_name, debug_run, task='rg'))
meta

In [ ]:
meta['hparams']

In [ ]:
load_metadata(RunId('0123_174651', debug=False, task='cls'))

## CMs

In [ ]:
# TODO: implement for multiple dataset_types at the same time (see function below)
# FIXME: update for RunId stuff

def plot_cms_multiple_runs(run_names, dataset_type='train', percentage=False):
    errors = defaultdict(list)

    n_runs = len(run_names)
    n_diseases = len(diseases)

    plt.figure(figsize=(n_runs*5, n_diseases * 3))

    for i_run, run_name in enumerate(run_names):
        results = load_metrics(run_name)      

        # for dataset_type in dataset_types:
        if dataset_type not in results:
            errors['dataset-type-not-found'].append(run_name)
            continue

        metrics = results[dataset_type]

        for i_disease, disease in enumerate(diseases):
            key = f'cm_{disease}'
            if key not in metrics:
                errors[f'{key}-not-in-metrics'].append(run_name)
                continue

            cm = metrics[key]

            subplt_index = n_runs * i_disease + i_run + 1
            plt.subplot(n_diseases, n_runs, subplt_index)

            title = shorten_run_name(run_name) if i_disease == 0 else None
            plot_cm(cm, title=title, percentage=percentage,
                    xlabel=(i_disease == n_diseases-1),
                    ylabel=False,
                   )
            plt.ylabel(disease)


        print(run_name)
    print(f'Dataset type: {dataset_type}')
    
    n_errors = sum(v for v in errors.values())
    if n_errors > 0:
        print('Errors: ', {k: len(v) for k, v in errors.items()})

In [ ]:
run_names = [
#     '0123_174651_cxr14_mobilenet-v2_lr0.0001_hint_normS_size256_sch-roc_auc-p5-f0.1_noes',
    '0125_211742_cxr14_densenet-121-v2_lr0.0001_hint_normS_size200_sch-roc_auc-p5-f0.1_noes',
#     '0128_092013_cxr14-clahe_densenet-121-v2_lr0.0001_normS_size256_sch-roc_auc-p5-f0.1_noes',
#     '0128_165641_cxr14-clahe_densenet-121-v2_lr0.0001_hint_normS_size200_sch-roc_auc-p5-f0.1_noes',
    '0129_150255_cxr14_densenet-121-v2_lr0.0001_hint_balance_normS_size200_sch-roc_auc-p5-f0.1_noes',
    '0129_150753_cxr14_densenet-121-v2_lr0.0001_balance_normS_size256_sch-roc_auc-p5-f0.1_noes',
]

In [ ]:
plot_cms_multiple_runs(run_names, 'test', percentage=True)

## Prediction distribution

### Functions

In [ ]:
def plot_distributions(run_ids, dataset_types='test', density=True):
    # FIXME: load this from the metadata?
    diseases = list(DISEASES)
    
    if isinstance(dataset_types, str):
        dataset_types = (dataset_types,)
    
    n_diseases = len(diseases)
    n_runs = len(run_ids)
    n_dtypes = len(dataset_types)
    
    n_rows = n_diseases
    n_cols = n_runs * n_dtypes
    
    plt.figure(figsize=((n_runs*n_dtypes)*5, n_diseases*3))
    
    # To set y_lim the same for horizontal neighbors:
    max_upper_y_by_disease = np.zeros(len(diseases))
    axes_by_disease = {i: [] for i in range(len(diseases))}
    
    for i_run, run_id in enumerate(run_ids):
        df = load_outputs_csv(run_id)
        metrics = load_metrics(run_id)
        
        print(run_id.name)
        
        # Print metrics
        for dataset_type in metrics.keys():
            sub_metrics = metrics[dataset_type]
            macro_roc_auc = sub_metrics.get('roc_auc', -1)
            print(f'\t{dataset_type}: roc-auc={macro_roc_auc:.2f}')
        
        for i_dtype, dataset_type in enumerate(dataset_types):
            sub_df = df[df['dataset_type'] == dataset_type]
            sub_metrics = metrics.get(dataset_type, {})
    
            upper_y_by_disease = []
            for i_disease, disease in enumerate(diseases):
                gt_col = f'{disease}-gt'
                pred_col = f'{disease}-pred'
                cols = [gt_col, pred_col]

                if gt_col not in sub_df.columns:
                    continue
                
                disease_df = sub_df[cols]

                pos_samples = disease_df[disease_df[gt_col] == 1][pred_col]
                neg_samples = disease_df[disease_df[gt_col] == 0][pred_col]

                # Plot
                subplt_index = (n_runs * n_dtypes) * i_disease + i_run * n_dtypes + i_dtype + 1
                plt.subplot(n_rows, n_cols, subplt_index)

                if i_disease == 0: plt.title(f'{run_id.short_name} ({dataset_type})')

                plt.hist(pos_samples, fc=(1, 0, 0, 0.4), label='pos', density=density)
                plt.hist(neg_samples, fc=(0, 0, 1, 0.4), label='neg', density=density)

                if i_disease == n_diseases-1: plt.xlabel('Prediction value')
                
                roc_auc = sub_metrics.get(f'roc_auc_{disease}')
                # plt.ylabel(f'{disease[:6]} ={roc_auc:.2f}', fontsize=16)
                plt.ylabel(disease, fontsize=16)
                plt.annotate(f'{roc_auc:.2f}', (0.5, 0.95),
                             xycoords='axes fraction',
                             fontsize=20, va='top', ha='center',
                            )

                # plt.legend()
                
                # To set ylims for horizontal neighbors:
                current_axes = plt.gca()
                _, upper_y = current_axes.get_ylim()
                upper_y_by_disease.append(upper_y)
                axes_by_disease[i_disease].append(current_axes)

            max_upper_y_by_disease = np.maximum(max_upper_y_by_disease, upper_y_by_disease)
            
            
    # Correctly set ylims
    for i_disease in range(len(diseases)):
        upper_y = max_upper_y_by_disease[i_disease]
        axes = axes_by_disease[i_disease]
        
        for ax in axes:
            ax.set_ylim(0, upper_y)

### Compare runs

In [ ]:
run_ids = [
# '1215_174443_cxr14_resnet-50-v2_lr0.0001_os_Cardiomegaly_normS_size256_sch-roc_auc-p5-f0.1',
# '0125_211742_cxr14_densenet-121-v2_lr0.0001_hint_normS_size200_sch-roc_auc-p5-f0.1_noes',
# '0201_180839_cxr14_tfs-small_lr0.0001_normS_size256_sch-roc_auc-p5-f0.1_noes',
# '0203_163918_cxr14_tiny-densenet_lr0.0001_gavg_aug-No-Finding-cls0_normS_size256_sch-roc_auc-p3-f0.1',
# '0203_185555_cxr14_tiny-densenet_lr0.0001_hint_gavg_normS_size256_sch-roc_auc-p3-f0.1',

# Small examples:
# '0222_230258',
# '0219_175036',
    
# Balance examples:
# '0201_180818_cxr14_tfs-small_lr0.0001_balance_normS_size256_sch-roc_auc-p5-f0.1_noes',
# '0201_143527_cxr14_mobilenet-v2_lr0.0001_noig_balance_normS_size256_sch-roc_auc-p5-f0.1_noes',

# Bad examples:
# '0202_164454_cxr14_tiny-densenet_lr0.0001_normS_size256_sch-roc_auc-p5-f0.1',
    
    # Good examples:
#     RunId('0317_110119', debug=False, task='cls'),
#     RunId('0318_154252', debug=False, task='cls'),
#     RunId('0406_003450', debug=False, task='cls-seg'),

    # Previous best:
    # RunId('0123_174651', debug=False, task='cls'),
    
    # Best CL:
    # RunId('0311_101753', debug=False, task='cls'),
    # RunId('0410_145310', debug=False, task='cls'),
    # RunId('0413_174536', debug=False, task='cls'),
    
    # Best CLS-SEG (better than CLS):
    RunId('0402_062551', debug=False, task='cls-seg'),
    # RunId('0415_032125', debug=False, task='cls-seg'), # augmentation
    RunId('0422_163242', debug=False, task='cls-seg'),
    
    # RunId('0422_002431', debug=False, task='cls-seg'),
    # RunId('0423_033202', debug=False, task='cls-seg'),
]

In [ ]:
plot_distributions(run_ids, ('train', 'val'))

# Covid examples

In [ ]:
LABELS = ['covid', 'non-c', 'normal']

## Example 1: Covid-UC, size 512, frontal only

In [ ]:
results = load_results('0717_120222_covid-x_densenet-121_lr1e-06_os_aug-covid', 'covid-uc_size512_frontal')
list(zip(*results))[0]

In [ ]:
result = results[0][1]
result

In [ ]:
cm = result['all']['cm']

In [ ]:
fig = plt.figure()
fig.patch.set_facecolor('white')

plot_cm(cm, labels=LABELS, title='Trained on Covid-X (frontal only)', percentage=False)

## Example 2: Covid-X + Covid-UC

In [ ]:
results = load_results('0722_201835_covid-uc_densenet-121_lr0.0001_os-max10_aug_pre-covid-x')
list(zip(*results))[0]

In [ ]:
cm = results[0][1]['val']['cm']

In [ ]:
fig = plt.figure()
fig.patch.set_facecolor('white')

plot_cm(cm, labels=LABELS, title='Trained on Covid-X + UC', percentage=False)

## Example 3

In [ ]:
results = load_results('0722_193534_covid-uc_densenet-121_lr0.0001_os-max10_aug_size256')
list(zip(*results))[0]

In [ ]:
cm = results[0][1]['val']['cm']

In [ ]:
fig = plt.figure()
fig.patch.set_facecolor('white')

plot_cm(cm, labels=LABELS, title='Trained on Covid-UC', percentage=False)

## Example 4: winner

In [ ]:
results = load_results('0717_120222_covid-x_densenet-121_lr1e-06_os_aug-covid')
list(zip(*results))[0]

In [ ]:
cm = results[0][1]['test']['cm']

In [ ]:
plot_cm(cm, labels=['covid', 'pneumonia', 'normal'], title='Covid-X', percentage=False)